### Homework 9: Serverless

In [26]:
import tensorflow as tf
from keras.preprocessing.image import load_img
from keras.applications.xception import preprocess_input
 

from io import BytesIO
from urllib import request
from PIL import Image

import numpy as np
import json
import os

In [16]:
tf.__version__

'2.18.0'

In [17]:
model_path = 'model_2024_hairstyle.keras'
model = tf.keras.models.load_model(model_path)

# Display the model architecture
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 198, 198, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 313632)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │    20,072,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,146,948 (153.15 MB)

 Trainable params: 20,073,473 (76.57 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 20,073,475 (76.57 MB)

In [18]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()


INFO:tensorflow:Assets written to: /tmp/tmpvcsyqzma/assets


INFO:tensorflow:Assets written to: /tmp/tmpvcsyqzma/assets


Saved artifact at '/tmp/tmpvcsyqzma'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  135201895459984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135201895460368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135201895460560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135201895461520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135201895461712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135201895462288: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1733998743.464780   66745 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1733998743.484122   66745 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2024-12-12 12:19:04.347070: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpvcsyqzma
2024-12-12 12:19:04.373749: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-12-12 12:19:04.373797: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpvcsyqzma
2024-12-12 12:19:04.787227: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-12-12 12:19:11.103975: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpvcsyqzma
2024-12-12 12:19:11.249208: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 6900456 microseconds.


In [19]:
with open('model_2024_hairstyle.tflite', 'wb') as f:
    f.write(tflite_model)


In [20]:
tflite_model_size = os.path.getsize('model_2024_hairstyle.tflite')
print(f'TFLite model size: {tflite_model_size / (1024 * 1024):.2f} MB')

TFLite model size: 76.58 MB


In [21]:
interpreter = tf.lite.Interpreter(model_path='model_2024_hairstyle.tflite')

# Allocate tensors
interpreter.allocate_tensors()

# Get input and output tensor details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Get output tensor
output_data = output_details[0]['index']


In [22]:
output_data

13

In [23]:
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [31]:
image = download_image('https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg')

In [33]:
img = prepare_image(image, target_size=(200,200))

In [ ]:
image_array = np.array(img)
image_array = image_array / 255.0 

first_pixel_r_channel = image_array[0, 0, 0]
first_pixel_r_channel

print(f"First pixel R channel value after pre-processing: {first_pixel_r_channel}")


First pixel R channel value after pre-processing: 0.23921568627450981


In [37]:
tflite_model_path = "model_2024_hairstyle.tflite"  
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

image = download_image('https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg')
image = prepare_image(image, target_size=(200,200))
image = image.resize((200, 200))  
image_array = np.array(image) / 255.0  
image_array = np.expand_dims(image_array, axis=0)  


interpreter.set_tensor(input_details[0]['index'], image_array.astype(np.float32))


interpreter.invoke()

output = interpreter.get_tensor(output_details[0]['index'])
print(f"TFLite model output: {output[0]}")


TFLite model output: [0.8934675]
